# WP12

## Theoretical Methods for Condensed Matter

### Tom Frömbgen

# Part II

# Theory of non-covalent interactions
---
## General Setup

In [17]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
import lmfit as lf
from lmfit.models import ExpressionModel
from inspect import signature
#from scipy.optimize import curve_fit
#from scipy.optimize import differential_evolution as DE
import os
import sys
import linecache
from pathlib import Path
from IPython.display import display_html, display
from itertools import chain, cycle


save_as_pgf = 1
save_as_png = 0

if save_as_pgf == 1:
    mpl.use("pgf")
    mpl.rcParams['text.usetex'] = True
    mpl.rcParams['pgf.texsystem'] = 'pdflatex'
    mpl.rcParams['pgf.rcfonts'] = False
    mpl.rcParams['pgf.preamble'] = r'\usepackage{amssymb} \usepackage{amsmath}'
    mpl.rcParams['font.family'] = 'serif'
    mpl.rcParams['font.serif'] = 'Computer Modern'
    mpl.rc('axes', titlesize=10)     # fontsize of the axes title
    mpl.rc('axes', labelsize=10)     # fontsize of the x and y labels
    mpl.rc('xtick', labelsize=8)    # fontsize of the tick labels
    mpl.rc('ytick', labelsize=8)    # fontsize of the tick labels
    mpl.rc('legend', fontsize=8)
    mpl.rc('figure', titlesize=10)   # fontsize of the figure title

    lw = 1 # width of grid lines

    # width of tex document in inches
    fig_width = 6.69423
    fig_height = fig_width * 0.6
    #import matplotlib.pyplot as plt
    
else:
    #import matplotlib.pyplot as plt
    plt.rc('font', size=16)          # controls default text sizes
    plt.rc('axes', titlesize=16)     # fontsize of the axes title
    plt.rc('axes', labelsize=18)     # fontsize of the x and y labels
    plt.rc('xtick', labelsize=16)    # fontsize of the tick labels
    plt.rc('ytick', labelsize=16)    # fontsize of the tick labels
    plt.rc('legend', fontsize=18)    # legend fontsize
    plt.rc('figure', titlesize=18)   # fontsize of the figure title

    lw = 1

    # height and width for nice print in jupyter notebook
    fig_width = 20
    fig_height = 10
    


# number of decimals printed in numpy stuff
np.set_printoptions(formatter={'float': '{: 0.3f}'.format})

# uni bonn colors
blue = "#07529a"
yellow = "#ffb90c"

# number of decimals
ytick_format = mpl.ticker.FormatStrFormatter('%.2f')

/usr/lib64/python3.6/site-packages/matplotlib/__init__.py:1405: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


## Helpers

In [18]:
def save_plot_as_pgf(fig, path):
  if save_as_pgf == 1:
    fig.savefig(path, format="pgf")


def save_plot_as_png(fig, path):
  if save_as_png == 1:
    fig.savefig(path, format="png")


def checkForFaultyData(df):
  if df.isnull().sum().sum() != 0:
    df.dropna(how='any', inplace=True)
    print(df.isnull().sum())
    print("WARNING! Dropped some null values.")
  return df

def display_side_by_side(*args,titles=cycle([''])):
    html_str=''
    for df,title in zip(args, chain(titles,cycle(['</br>'])) ):
        html_str+='<th style="text-align:center"><td style="vertical-align:top">'
        html_str+=f'<h2>{title}</h2>'
        html_str+=df.to_html().replace('table','table style="display:inline"')
        html_str+='</td></th>'
    display_html(html_str,raw=True)
    
eh2kcal = 627.503 
ev2kcal = 23.0609
eh2kj = 2625.5 
ev2kj = 96.4869

# Main part

# 5. Bulk Properties of NaCl
## 5.2 Structural relaxations on NaCl

In [42]:
def getFileList():
    filelist = []
    for file in sorted(Path("./ex1/opts").rglob("*.out")):
        filelist.append(file)

    if len(filelist) == 0:
        sys.exit("No '*data' files found.")

    return filelist


data = []
flist = getFileList()
linestoread=[2]

for i, file in enumerate(flist):
    with open(file) as f:
        for j, line in enumerate(f):
          if "    TOTAL CPU TIME =" in line:
            time = float(line.split()[4])
          if "         A              B              C           ALPHA      BETA       GAMMA" in line:
            a = float(next(f).split()[1])
            

    data.append([file.parent, time, a])
  
cols = ["Functional", "CPU time", "a"]
df = pd.DataFrame(columns=cols, data=data)
print(df)

# save to csv
save_loc = os.path.join(os.path.realpath("."), "ex1/nacl-relax.csv")
df.to_csv(save_loc, index=False)

         Functional  CPU time         a
0  ex1/opts/lda_vwn   182.140  5.410157
1     ex1/opts/m06l   187.466  5.491994
2     ex1/opts/pbe0   971.877  5.573240
3    ex1/opts/pwgga   195.352  5.614326


# 6. Calculating Surfaces of TiO2